# Limpiar Coordenadas para Earthquake

## Importar librerías

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import re
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
from time import sleep

In [5]:
geolocator = Nominatim(user_agent="MozillaFirefox", timeout=10)

## Cargar base de datos

03desastreslimpio.csv

In [6]:
df = pd.read_csv('../../Data/03Limpio/03desastreslimpio.csv')
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '../../Data/03Limpio/03desastreslimpio.csv'

## EDA Inicial Básico

In [ ]:
df.shape

In [ ]:
df.info()

## Filtro Disaster Type == Earthquake

In [ ]:
filter = df['Disaster Type'] == 'Earthquake'
df_earthquake = df[filter]

In [ ]:
df_earthquake.head(2)

In [ ]:
df_earthquake.shape

In [ ]:
df_earthquake['Disaster Type'].dtype

# Limpieza de coordenadas

## Función para limpiar coordenadas filtrado para df_earthquake

In [ ]:
# Initializar geolocator
geolocator = Nominatim(user_agent="eqlimpiarcoord")

# Convertir 'Latitude' y 'Longitude' a string
df_earthquake['Latitude'] = df_earthquake['Latitude'].astype(str)
df_earthquake['Longitude'] = df_earthquake['Longitude'].astype(str)

# Limpiar coordenadas de latitude y longitude
df_earthquake['Latitude'] = df_earthquake['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x))
df_earthquake['Longitude'] = df_earthquake['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x))

# Remover puntos finales
df_earthquake['Latitude'] = df_earthquake['Latitude'].str.rstrip('.')
df_earthquake['Longitude'] = df_earthquake['Longitude'].str.rstrip('.')

# Añadir valores anómalos de latitud y longitud a listas
anomalous_lat = []
anomalous_lon = []

# Función para convertir coordenadas y detectar anomalías
def convert_coordinates(x, convert_nan=True):
    if not x:
        return np.nan

    # Remover caracteres 'N' y 'E'
    x = x.replace(' N', '').replace(' E', '')

    # Remover puntos extra en decimales
    x = re.sub('^(\d+\.\d{2})\..*', r'\1', x)

    # Conversión a negativo para S y W
    try:
        value = float(x)
        if x[-1] == 'S' or x[-1] == 'W':
            return -value
        else:
            return value
    except ValueError:
        return np.nan

# Convertir a float
df_earthquake['Latitude'] = df_earthquake['Latitude'].apply(convert_coordinates)
df_earthquake['Longitude'] = df_earthquake['Longitude'].apply(convert_coordinates)

# Redondear decimales
df_earthquake['Latitude'] = df_earthquake['Latitude'].round(2)
df_earthquake['Longitude'] = df_earthquake['Longitude'].round(2)

# Identificar valores anómalos de latitude y longitude
for index, row in df_earthquake.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']

    if latitude < -90 or latitude > 90:
        anomalous_lat.append(index)

    if longitude < -180 or longitude > 180:
        anomalous_lon.append(index)

# Revisar inconsistencias de cantidad entre anómalos de latitude y longitude
if len(anomalous_lat) != len(anomalous_lon):
    print("Valores inconsistentes de latitud y longitud.")
    # Gestionar la inconsistencia, como remover los valores extras o ajustar las listas
    # Por ejemplo, remover los valores extra de latitud:
    anomalous_lat = anomalous_lat[:len(anomalous_lon)]
    print("Longitud de latitud anómala ajustada:", len(anomalous_lat))

# Crear nuevo DataFrame con coordenadas limpias y emparejadas
df_eqcleaned = df_earthquake.copy()

# Función para rellenar coordenadas anómalas usando geocoding
def fill_anomalous_coordinates(row):
    if row.name in anomalous_lat and row.name in anomalous_lon:
        try:
            location = geolocator.reverse((row['Latitude'], row['Longitude']), timeout=10)
            if location and location.latitude is not None and location.longitude is not None:
                row['Latitude'] = location.latitude
                row['Longitude'] = location.longitude
                row['Location'] = location.address
                row['Country'] = location.raw['address'].get('country')
        except:
            pass
    return row

# Rellenar coordenadas anómalas
df_eqcleaned = df_eqcleaned.apply(fill_anomalous_coordinates, axis=1)

# Borrar filas con location y country no emparejadas
df_eqcleaned = df_eqcleaned[~((df_eqcleaned['Latitude'].isin(anomalous_lat)) & (df_eqcleaned['Longitude'].isin(anomalous_lon)))]

# Borrar filas con valores null o nan en columnas de Latitud y Longitud
df_eqcleaned = df_eqcleaned.dropna(subset=['Latitude', 'Longitude'])

# Convertir Latitud y Longitud a float con 2 decimales
df_eqcleaned['Latitude'] = df_eqcleaned['Latitude'].astype(float).round(2)
df_eqcleaned['Longitude'] = df_eqcleaned['Longitude'].astype(float).round(2)

# Exportar el DataFrame limpio a archivo
df_eqcleaned.to_csv('earthquakefilteredcleanedgeoloc.csv', index=False)


1. Definimos la función fill_anomalous_latlon que toma una fila como entrada. Si los valores de latitud y longitud de la fila se encuentran en las listas anomalous_lat y anomalous_lon, respectivamente, la función intenta geocodificar la ubicación mediante la columna Location o la columna Country. Si se encuentra una ubicación válida, se actualizan los valores de latitud y longitud de la fila.

2. La llamada a la función sleep(1) agrega un retraso de 1 segundo entre las solicitudes de geolocalización para cumplir con la política de uso del servicio de geocodificación.

3. Luego creamos un objeto geolocalizador utilizando el geocodificador Nominatim de la biblioteca geopy.

4. Finalmente, aplicamos la función fill_anomalous_latlon a cada fila del DataFrame usando el método apply con axis=1, lo que indica que la función debe aplicarse por filas.

## Función con geopy para limpiar coordenadas

## Tests de verificación de limpieza de coordenadas

In [ ]:
df_earthquake[['Latitude', 'Longitude', 'Location', 'Country']][700:800:]

In [ ]:
# Filtrar DataFrame para detectar valores anómalos de latitud y longitud
df_eqanomalous = df_earthquake[
    df_earthquake['Latitude'].isin(anomalous_lat) | df_earthquake['Longitude'].isin(anomalous_lon)
]

# Crear pivot table para comparar las columnas de latitude, longitude, location, y country
df_eqanomalous_pivot = df_eqanomalous.pivot_table(
    index=['Location', 'Country'],
    values=['Latitude', 'Longitude'],
    aggfunc='first'
)

# Mostrar la pivot table
print(df_eqanomalous_pivot)


In [ ]:
print(df_earthquake['Latitude'])
print(df_earthquake['Longitude'])

In [ ]:
print(df_earthquake['Latitude'].dtype)
print(df_earthquake['Longitude'].dtype)

In [ ]:
# # Examinar filas específicas para verificar si el proceso de limpieza ha manejado los valores anómalos correctamente.
# print(df_earthquake.loc[6000, 'Latitude'])
# print(df_earthquake.loc[6000, 'Longitude'])

In [ ]:
# Valores nulos de Longitude y Latitude
df_earthquake[['Longitude', 'Latitude']].isnull().sum()

## Valores Anómalos de Latitud y Longitud

In [ ]:
len(anomalous_lat)

In [ ]:
len(anomalous_lon)

#### Índices Anómalos de Latitud y longitud

In [ ]:
anomalous_lat_idx = []
for index, lat in enumerate(anomalous_lat):
    if lat < -90 or lat > 90:
        anomalous_lat_idx.append(index)

print("Índices Anómalos de Latitud:", anomalous_lat_idx)

In [ ]:
anomalous_lon_idx = []
for index, lon in enumerate(anomalous_lon):
    if lat < -90 or lat > 90:
        anomalous_lon_idx.append(index)

print("Índices Anómalos de Longitud:", anomalous_lon_idx)

## Outliers de Latitude y Longitude

In [ ]:
sns.boxplot(df_earthquake['Latitude'])

In [ ]:
sns.boxplot(df_earthquake['Longitude'])

## Valores faltantes

In [ ]:
df_earthquake.info()

In [ ]:
df_earthquake.isnull().sum()

# Visualizaciones

## Heatmaps

# Pivot Tables

In [ ]:
df_earthquake_origin_geolocation_pivot = df_earthquake_filled_latlon.pivot_table(
    index='Origin',
    columns='Disaster Subtype',
    values=['Latitude', 'Longitude', 'Location'],
    aggfunc='sum',
    fill_value=0
)
df_earthquake_origin_geolocation_pivot